# 일일 업데이트시 다수의 키워드를 크롤링 후 DB에 업데이트하는 로직

In [3]:
from requests import get, post, Session
from urllib import parse
import json
from datetime import datetime, timedelta
import emoji
import time
import pymysql
import random

# DB 접속 정보

In [4]:
host = 'hotkey.czighwjgpkkz.ap-northeast-2.rds.amazonaws.com'
user = 'admin'
pw = 'hotkey123'
port = 3306

# Account 정보 & cur_idx => 현재 이용하고 있는 계정

In [7]:
# 전체 계정 (사실상 전부 차단 전력있음)
total_acc = [('kj10522002@korea.ac.kr','kj76081460!'), ('quickpass8@gmail.com','thskrl1!'),('hotkey2','gktzl2'),
                 ('seungirumd+1@gmail.com', 'pinstaw25'), ('lshyun0510.11@gmail.com', '*thvmxmdnpdj'),
                 ('seyun1052dev@gmail.com', 'kj76081460!'), ('lshyun0510.12@gmail.com', '*thvmxmdnpdj'),
                 ('quickpass88@gmail.com', 'thskrl1!'), ('hotkey22', 'gktzl22!'), ('tamikia0@hbviralbv.com', 'cvd0^ktm')]

# 사용중인 계정 리스트
using_acc = [(i[0], i[1], datetime.now()) for i in total_acc]
# 임시 테스트용..
# using_acc = [('kj10522002@korea.ac.kr','kj76081460!'), ('tamikia0@hbviralbv.com','cvd0^ktm')]

In [8]:
#최종 로그인 날짜에 따라 sorting하는 logic
using_acc.sort(key=lambda x:x[2]) 

### 그 외 초기값..

In [9]:
delimiter = 'HOTKEY123!@#'
cur_idx = -1 #account정보 가르킨다.

## 기존 데이터 중 update가 필요한 wordlist를 생성하는 함수 (ref = 몇일 이상 지난 tag를 업데이트 할지, 정책에 따라 바꾸기) - return uplist
#### 실제 사용은 파일 형태가 아니라 DB에서 뽑아오는 로직으로 변경 필요.

In [11]:
def update_check(ref = 2):
    conn = pymysql.connect(host=host, user=user, password=pw, port=port, charset='utf8') #db연결
    cur = conn.cursor(pymysql.cursors.DictCursor)
    cur.execute('use hotkey;')
    cur.execute('SELECT tid,tname FROM (SELECT tid, tname, DATEDIFF(NOW(),up_date) AS diff FROM is_tag NATURAL JOIN tag_info) AS a WHERE diff >= (%s);', (ref))
    res = cur.fetchall()
    uplist = list()
    for row in res:
        uplist.append(row['tname'])
    return uplist

In [14]:
# 사용 예시
update_check(3), update_check(2)

([],
 ['일상',
  '일상스타그램',
  '데일리그램',
  '데일리',
  '인스타일상',
  '오오티디',
  'ootd',
  '오늘',
  '어제',
  '실시간',
  '인스타굿',
  '주말',
  '풍경',
  '풍경스타그램',
  '일상소통그램',
  '선팔',
  '맞팔',
  '팔로우',
  '인친',
  '사진스타그램',
  '데일리스타그램',
  '소통',
  '소통해요',
  '소통환영',
  '데일리룩',
  '일기장',
  '하루일과',
  '그냥',
  '휴무',
  '오후',
  '오랜만',
  '주말그램',
  '주말일상',
  '일요일',
  '굿모닝',
  '월요일',
  '월요병',
  'daily',
  '심심',
  '밤산책',
  '토요일',
  '저녁밥상',
  '2021',
  '여자친구',
  '남자친구',
  '럽스타그램',
  '럽스타',
  '데이트',
  '연애중',
  '세젤예',
  '남친스타그램',
  '여친스타그램',
  '행복한하루',
  '내사랑',
  '남친몬',
  '여친몬',
  '곰신',
  '곰신스타그램',
  '러브러브',
  '알라뷰',
  '사랑중',
  '내꺼',
  '연애하고싶다',
  '연애상담',
  'n년차부부',
  'n년차커플',
  '커플계정',
  '연상연하커플',
  '데이트코스',
  '데이트스타그램',
  '사랑꾼',
  '데이트그램',
  '커플',
  '커플샷',
  '기념일',
  '사랑해',
  '연애',
  '사랑',
  '연애그램',
  '사랑해요',
  '추억',
  '포옹샷',
  '여친짤',
  '여친선물',
  '여친룩',
  '남친룩',
  '연애일기',
  '연인',
  '부부',
  '부부스타그램',
  '남편',
  '와이프',
  '브라이덜샤워',
  '결혼준비',
  '웨딩촬영',
  '웨딩사진',
  '알콩달콩',
  '여행',
  '어디까지가봤니',
  '여행에미치다',
  '여행지추천',
  '떠나고싶다',
  '드라이

# 새로 업데이트된 트렌드 태그 셋을 받아오는 함수 (구현 예정)
### 기존 DB와 겹치는 것이 있다면, 따로 update list에 올려야한다. trend set이 아니라
### trendset은 'insert문' // update list는 'update 문'으로 관리.

In [15]:
def trend_check():
    trendlist = ()
    ###미구현###
    ###여기에 구현 예정###
    return trendlist

## 최근 게시물 수집 함수

In [ ]:
def hot_key_instagram_recent(query, headers, session, max_page=40): #return (T/F, recent_list, timestamp, image_list)
    postcnt = 0 #포스트 수 카운팅, 300개 되면 stop.
    timestamp = 0
    
    before = datetime.now()
    recent_list = list()
    image_list = list()
    
    url = 'https://i.instagram.com/api/v1/tags/web_info/?tag_name={}'.format(parse.quote(query))
    headers['x-csrftoken'] = session.cookies.get('csrftoken') #재설정
    headers['referer'] = 'https://www.instagram.com/explore/tags/{}/'.format(parse.quote(query))
    resp = session.get(url, headers = headers)
    
    if(resp.status_code == 404):
        print('검열되는 해시태그이거나 해당 태그를 담은 페이지가 존재하지 않습니다.')
        print('response code : ', 404)
        return (False, recent_list, timestamp, image_list)   
    
    elif(resp.status_code != 200): #응답 에러
        print('response error during crawling : please retry with some other way')
        print('response code : ', resp.status_code)
        #print(resp.text)
        return (False, recent_list, timestamp, image_list) #계정바꿔서 재시도

    
    #첫페이지 응답 정상인경우
    
    ################################################# 비정상입력 예외처리
    try:
        resource = resp.json()
    except:
        print("응답 코드는 정상이나, 비정상적인 데이터 형식 반환됨")
        print(resp.text)
        return (False, recent_list, timestamp, image_list)
    #################################################
    
    #recent 돌기
    
    #print('\n최근 게시물: 1페이지....')
    recent_list.append([])
    image_list.append([])
    for i in resource['data']['recent']['sections']: #0~8
        for k in i['layout_content']['medias']:
            if (k['media']['caption'] != None):
                if 'text' in k['media']['caption'].keys():
                    recent_list[0].append(emoji.demojize(k['media']['caption']['text'])) # 이모티콘 제거 ###########################
                    timestamp = max(timestamp, k['media']['caption']['created_at_utc']) # 최신 포스트의 현재 시간 1669098438 ##############################
                    ###--------------------------1123 추가
                    #######################################################################
                    try:
                        image_list[0].append(k['media']['image_versions2']['candidates'][0]['url']) # 포스트의 이미지 URL 
                    except:
                        image_list[0].append(k['media']['carousel_media'][0]['image_versions2']['candidates'][0]['url'])
                        # 복수의 이미지가 있는 포스트의 첫이미지 URL 
                    #########################################################
                    postcnt+=1
    print('누적 게시물 수 :', postcnt)
    time.sleep(random.uniform(1,5))
                    
    #recent기준 다음 페이지 관련 정보
    recent_info = {'max_id':'', 'page': '', 'isnext': False}
    if resource['data']['recent']['more_available']:
        recent_info['max_id'], recent_info['page'], recent_info['isnext'] = \
        resource['data']['recent']['next_max_id'], resource['data']['recent']['next_page'], True
        
    #그 다음 페이지부터 recent 돌기
    url = 'https://i.instagram.com/api/v1/tags/{}/sections/'.format(parse.quote(query))
    data = {'max_id': recent_info['max_id'], 'page': recent_info['page'], 'surface' : 'grid', 'tab': 'recent'}
    
    while (recent_info['isnext']==True and data['page']<max_page):
        #print('\n최근 게시물: {}페이지....'.format(data['page']+1))
        headers['x-csrftoken'] = session.cookies['csrftoken'] #매번 재설정해주기
        resp = session.post(url, data=data, headers=headers)
        ########################################################################### 스크롤 시작 이후 로그인 소요 발생 시 예외 처리
        if(resp.status_code != 200): #응답 에러
            print('response error during crawling : please retry with some other way\n')
            print('response code : ', resp.status_code)
            #print(resp.text)
            return (False, recent_list, timestamp, image_list) #계정바꿔서 재시도
        ##########################################################################
        ################################################# 비정상입력 예외처리
        try:
            resource = resp.json()
        except:
            print("응답 코드는 정상이나, 비정상적인 데이터 형식 반환됨")
            return (False, recent_list, timestamp, image_list)
        #################################################
        recent_list.append([])
        image_list.append([])
        for i in resource['sections']: #0~8
            for k in i['layout_content']['medias']: #0~2
                if (k['media']['caption'] != None):
                    if 'text' in k['media']['caption'].keys():
                        recent_list[data['page']].append(emoji.demojize(k['media']['caption']['text'])) # 이모티콘 제거 ###############
                        ###--------------------------1123 추가
                        #######################################################################
                        try:
                            image_list[data['page']].append(k['media']['image_versions2']['candidates'][0]['url']) # 포스트의 이미지 URL 
                        except:
                            image_list[data['page']].append(k['media']['carousel_media'][0]['image_versions2']['candidates'][0]['url'])
                            # 복수의 이미지가 있는 포스트의 첫이미지 URL 
                        #########################################################
                        postcnt+=1
                        if (postcnt >= 300):
                            print('누적 게시물 수 :', postcnt)
                            print('최근 게시물 수집 완료!')
                            print('총 소요시간 : ', datetime.now() - before,'\n\n')
                            return (True, recent_list, timestamp, image_list)
            
        if resource['more_available']:
            data['max_id'] = resource['next_max_id']
            data['page'] = resource['next_page']
    
        else:
            recent_info['isnext']=False
        print('누적 게시물 수 :', postcnt)
        time.sleep(random.uniform(1,5))
    
    print('누적 게시물 수 :', postcnt)
    print('최근 게시물 수집 완료!')
    print('총 소요시간 : ', datetime.now() - before,'\n')
    return (True, recent_list, timestamp, image_list)

# 현재까지 not_colleccted : ['['꿀주말 ', '운동스타그램', '러닝 ', '인테리어']]
# corpus가 ' '인경우, 404떠서 수집 안된경우(아예 검열되는거), 
# 계정 막혀서 중간에 checkpoint 필요하다고 뜨는경우 (이 경우 중간에 그 계정을 acc_list에서 빼야함)
